[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/soloyant/deltax_workshop_2024/blob/main/tutorials/1_HydrodynamicModeling_ANUGA/[4]ANUGA_output_analysis.ipynb)
# I - Import packages and define the working directories

In [ ]:
import sys

if 'google.colab' in sys.modules:
  # In case the notebook is opened in google collab, here we download/install all the files we need behind the scenes
  try:
      import os
      os.chdir('/content')
      # Grab workbook files into colab directory
      !git clone https://github.com/soloyant/deltax_workshop_2024.git
      # Install everything using some bash scripts
      !/bin/bash /content/deltax_workshop_2024/tutorials/1_HydrodynamicModeling_ANUGA/utils/anuga_tools/install_anuga_colab.sh
      os.chdir('/content/deltax_workshop_2024/tutorials/1_HydrodynamicModeling_ANUGA/')
  except:
      pass

In [ ]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import netCDF4 as nc
import matplotlib.tri as tri
import geopandas as gpd
import utm

from utils import data_processing_tools as dpt
from utils import data_visualization_tools as viz
from utils.anuga_tools import anuga_tools as at
from model_inputs.model_settings import *

# Define the path to scripts and data
workshop_dir = os.getcwd()
# # Alternatively:
# workshop_dir = '/path/to/1_HydrodynamicModeling_ANUGA'
data_dir = os.path.join(workshop_dir, 'data')
if 'google.colab' in sys.modules:
    data_dir = os.path.join(data_dir, 'example_data')
model_inputs_dir = os.path.join(workshop_dir, 'model_inputs')
if 'google.colab' in sys.modules:
    model_inputs_dir = os.path.join(model_inputs_dir, 'example_data')
model_outputs_dir = os.path.join(workshop_dir, 'model_outputs')
model_visuals_dir = os.path.join(workshop_dir, 'visuals')
model_validation_dir = os.path.join(workshop_dir, 'validation')

for d in [model_inputs_dir, model_outputs_dir, model_visuals_dir, model_validation_dir]:
    Path(d).mkdir(parents=True, exist_ok=True)
        

# II - Load the saved model output

In [ ]:
# First import the domain generated during the DEM processing step
f_domain = os.path.join(data_dir, 'dxws_domain.shp')
domain_gdf = gpd.read_file(f_domain)
domain_polygon = np.asarray(domain_gdf.geometry[0].boundary.xy).T

f_anuga_output = os.path.join(model_outputs_dir, f"{model_name}.sww")
saved_output = nc.Dataset(f_anuga_output)

time = saved_output['time'][:].data
x = saved_output['x'][:].data+saved_output.xllcorner
y = saved_output['y'][:].data+saved_output.yllcorner
friction = saved_output['friction'][:]
N = time.shape[0]
Npt = x.shape[0]

triangles = saved_output['volumes'][:,:].data
triang = tri.Triangulation(x, y, triangles)

print('The loaded model includes '+ str(N)+ ' timesteps and ' + str(Npt) + ' triangles')

In [ ]:
# Recreate the tide function used for running the model from information saved in the model_setting.py file
tide_function = at.GenerateTideGauge(filename = f_tides,
                                      t_start = sim_starttime, 
                                      t_end = sim_endtime, 
                                      t_step = sim_timestep,
                                      offset = -.36,
                                      smoothing = False,
                                      smoothing_span = .1,
                                      hot_start = False,
                                      last_frame = 0)

# Convert it into a time series
t = (sim_time-sim_time[0]).astype('timedelta64[s]').astype(float)
tide_ts = [tide_function(i) for i in t]


# III - Water Level Animation
The function below will create an animation showing the spatio-temporal evolution of water elevations. The white "X" marks the location of the input tide gauge.

In [ ]:
# [OPTIONAL] Path to the background imagery and saved video file.
f_bg_img_tif = os.path.join(data_dir, 'misissippi_S2_background_image.tif')
f_anim_wl = os.path.join(model_visuals_dir, os.path.split(f_anuga_output)[1]).replace('.sww', '_water_level.mp4')

viz.animate_stage(saved_output, domain_polygon, 
                  gauge_location_ind=tide_gauge_ID[2:], 
                  gauge_time=sim_time, 
                  gauge_stage=tide_ts, 
                  bg_img_filepath=None, # Using a background significantly increases the processing time
                  fps = 20, 
                  initial_frame = 0, 
                  final_frame = N, 
                  framestep = 1, 
                  dpi=100, 
                  save=False, 
                  filename=f_anim_wl)

# IV - Velocity animation
We will now create an animation of the velocity with arrows showing the current direction. Displaying an arrow in every cell would make the figure overly busy. Instead, we will only plot arrows at the location of centerlines.

In [ ]:
%%capture --no-display

# Define file paths
f_centerlines_shp = os.path.join(data_dir, 'mississippi_River_Centerlines_V1/mississippi_River_Centerlines_V1.shp')
f_centerlines_buffered_shp = f_centerlines_shp.replace('.shp', '_buffered.shp')
f_centerlines_buffered_tif = f_centerlines_buffered_shp.replace('.shp', '.tif')

# Read and format centerline dataset
centerlines_gdf = gpd.read_file(f_centerlines_shp).to_crs(epsg=32615)

# Resample the centerlines using a regular footstep
centerlines_gdf = dpt.resample_gdf(centerlines_gdf, d=1000)

# Remove all centerlines located out of our model domain
centerlines_gdf = centerlines_gdf.overlay(domain_gdf)

# Create a trifinder object to find the triangle index
trifinder = triang.get_trifinder()

# Find the triangle index that contains the point
centerlines_gdf['triangle_index'] = trifinder(centerlines_gdf.centroid.x.to_numpy(), centerlines_gdf.centroid.y.to_numpy())

# Remove all points thate are not located within a triangle (i.e., labeled -1)
centerlines_gdf = centerlines_gdf.where(centerlines_gdf['triangle_index']!=-1).dropna().reset_index(drop=True)

# Create a list of unique querry points where to evaluate water velocity and display an arrow
querry_tri_inds = centerlines_gdf.triangle_index.unique()
querry_xy_inds = triang.triangles[querry_tri_inds,:]

Now that we identified for which triangles an arrow should be displayed, we can run the animate function.

In [ ]:
%%capture --no-display

# [OPTIONAL] Path to the background imagery and saved video file.
f_bg_img_tif = os.path.join(data_dir, 'misissippi_S2_background_image.tif')
f_anim_velo = os.path.join(model_visuals_dir, os.path.split(f_anuga_output)[1]).replace('.sww', '_water_velocity.mp4')

viz.animate_velocity(saved_output, domain_polygon, 
                     vcolormin=0,
                     vcolormax=1,
                     colormap='turbo',
                     gauge_location_ind=tide_gauge_ID[2:], 
                     querry_xy_inds=querry_xy_inds,
                     gauge_time=sim_time, 
                     gauge_stage=tide_ts, 
                     bg_img_filepath=None,  # Using a background significantly increases the processing time
                     fps = 20, 
                     initial_frame = 0, 
                     final_frame = N, 
                     framestep = 1, 
                     dpi=100, 
                     save=False, 
                     filename=f_anim_velo)